# Parametrized circuits

This tutorial shows you how to get started with the `povm_toolbox`.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from povm_toolbox.library.pm_sim_implementation import ClassicalShadows
from povm_toolbox.post_processor.povm_post_processor import POVMPostprocessor
from povm_toolbox.sampler.povm_sampler import POVMSampler
from qiskit.circuit.library import RealAmplitudes
from qiskit.quantum_info import (
    Operator,
    SparsePauliOp,
    Statevector,
    random_hermitian,
)
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit_ibm_runtime import Session

## Random Circuit

We first llok at a random the 2-qubit circuit, with depth 3.

In [3]:
# Prepare inputs.
n_qubit=2
qc = RealAmplitudes(num_qubits=n_qubit, reps=2)
H1 = SparsePauliOp.from_list([("II", 1), ("IZ", 2), ("XI", 3)])
theta = [[0, 1, 1, 2, 3, 5], [0, 1, 1, 2, 2, 5]]
qc.draw()

┌────────────────────────────────────────────────┐
q_0: ┤0                                               ├
     │  RealAmplitudes(θ[0],θ[1],θ[2],θ[3],θ[4],θ[5]) │
q_1: ┤1                                               ├
     └────────────────────────────────────────────────┘

We also draw some random observables 

In [4]:
set_observables = [SparsePauliOp.from_operator(random_hermitian(2**n_qubit)) for _ in range(10)]

For reference, we compute the true final state and the exact expectation values for the different observables.

In [5]:
exp_val = np.zeros((len(theta), len(set_observables)), dtype=complex)
for i, vals in enumerate(theta):
    exact_state = Statevector(np.array(Operator(qc.assign_parameters({key:val for key, val in zip(qc.parameters, vals)})).data[:,0]).squeeze())
    exp_val[i] = np.array([exact_state.expectation_value(obs) for obs in set_observables])
exp_val = np.real_if_close(exp_val)

## Classical Shadows

We now look at the implementation of Classical Shaodws measurement

In [6]:
# By default, the Classical Shadows (CS) measurement uses X,Y,Z measurements with equal probability.
cs_implementation = ClassicalShadows(n_qubit=n_qubit)
# Define the default shot budget.
cs_shots = 4096

In [7]:
pubs = [(qc, theta, np.random.randint(1000,10000) if np.random.randint(2)==0 else None, cs_implementation) for _ in range(5)]
print(pubs)

[(<qiskit.circuit.library.n_local.real_amplitudes.RealAmplitudes object at 0x10a159450>, [[0, 1, 1, 2, 3, 5], [0, 1, 1, 2, 2, 5]], 7487, <povm_toolbox.library.pm_sim_implementation.ClassicalShadows object at 0x10a158520>), (<qiskit.circuit.library.n_local.real_amplitudes.RealAmplitudes object at 0x10a159450>, [[0, 1, 1, 2, 3, 5], [0, 1, 1, 2, 2, 5]], 1487, <povm_toolbox.library.pm_sim_implementation.ClassicalShadows object at 0x10a158520>), (<qiskit.circuit.library.n_local.real_amplitudes.RealAmplitudes object at 0x10a159450>, [[0, 1, 1, 2, 3, 5], [0, 1, 1, 2, 2, 5]], None, <povm_toolbox.library.pm_sim_implementation.ClassicalShadows object at 0x10a158520>), (<qiskit.circuit.library.n_local.real_amplitudes.RealAmplitudes object at 0x10a159450>, [[0, 1, 1, 2, 3, 5], [0, 1, 1, 2, 2, 5]], None, <povm_toolbox.library.pm_sim_implementation.ClassicalShadows object at 0x10a158520>), (<qiskit.circuit.library.n_local.real_amplitudes.RealAmplitudes object at 0x10a159450>, [[0, 1, 1, 2, 3, 5], [0

In [8]:
# Run the sampler job locally using AerSimulator.
# Session syntax is supported but ignored because local mode doesn't support sessions.
backend = AerSimulator()
with Session(backend=backend) as session:
    # First define a standard sampler (that will be used under the hood).
    runtime_sampler = Sampler(session=session)
    # Then define the POVM sampler, which takes BaseSampler as an argument.
    sampler = POVMSampler(runtime_sampler)
    # Submit the job by specifying which POVM to use, which circuit(s) to measure and the shot budget.
    cs_job = sampler.run(pubs, shots=cs_shots)

/Users/dao/Documents/Internship/ic-povms/.venv/lib/python3.10/site-packages/qiskit_ibm_runtime/session.py:157: UserWarning: Session is not supported in local testing mode or when using a simulator.
  warnings.warn(


### Results
We retrieve the result object, which contains the POVM used and from which we can query the counts of each outcome.

In [9]:
cs_result = cs_job.result()
print(f"POVM: {cs_result[0].povm_metadata}")
print(f"Counts: {cs_result[1].get_counts(1)}")
cs_result[0].data.povm_measurement_cr.get_counts((1,0))

POVM: RandomizedPMsMetadata(povm=<povm_toolbox.library.pm_sim_implementation.ClassicalShadows object at 0x10a158520>, pvm_keys=[(0, 0), (0, 1), (2, 1), (1, 0), (0, 0), (0, 1), (0, 0), (2, 2), (2, 0), (0, 2), (2, 2), (0, 1), (2, 0), (0, 2), (2, 0), (0, 1), (0, 1), (1, 1), (0, 2), (2, 1), (1, 1), (2, 2), (1, 0), (1, 2), (1, 2), (1, 0), (1, 0), (1, 1), (1, 0), (2, 2), (2, 0), (1, 1), (2, 2), (1, 0), (1, 0), (0, 2), (1, 0), (2, 0), (2, 2), (2, 2), (2, 0), (0, 2), (0, 0), (2, 0), (2, 2), (0, 2), (0, 1), (2, 0), (2, 1), (2, 0), (0, 1), (2, 1), (0, 0), (1, 2), (0, 0), (0, 0), (1, 0), (1, 1), (2, 2), (2, 2), (1, 0), (2, 1), (2, 1), (0, 0), (1, 1), (2, 1), (0, 1), (2, 2), (0, 1), (1, 0), (1, 0), (0, 1), (1, 1), (0, 2), (2, 2), (0, 0), (2, 2), (0, 0), (1, 0), (1, 0), (0, 1), (0, 1), (2, 0), (0, 0), (1, 2), (1, 1), (2, 1), (0, 0), (1, 0), (1, 2), (0, 2), (1, 0), (0, 1), (0, 2), (2, 2), (2, 0), (2, 2), (0, 2), (2, 2), (0, 0), (0, 2), (2, 0), (2, 2), (2, 0), (1, 0), (1, 2), (0, 1), (0, 1), (1, 0), 

{'11': 1}

We now define our POVM post-processor, which will use the resukt object to estimate expectation values of some observables.

In [10]:
for i_state in range(len(theta)):
    for k in range(len(cs_result)):
        cs_postprocessor = POVMPostprocessor(cs_result[k])
        cs_est_exp_val = [cs_postprocessor.get_expectation_value(obs) for obs in set_observables]
        cs_shots = sum(cs_postprocessor.counts[0].values())

        n = int(np.ceil(np.log10(cs_shots)))
        print('Exact        CS')
        print(f'             ({cs_shots} shots)')
        for i in range(len(set_observables)):
            print(f'{np.real(exp_val[i_state,i]):>10.3e}   {cs_est_exp_val[i][i_state]:>{8+n}.3e}   {abs(cs_est_exp_val[i][i_state]-np.real(exp_val[i_state,i]))/abs(np.real(exp_val[i_state,i])):>{12-n}.1%}')

Exact        CS
             (7487 shots)
 6.918e-02      9.314e-02      34.6%
-1.364e+00     -1.394e+00       2.2%
 1.835e+00      1.901e+00       3.6%
-6.111e-01     -5.888e-01       3.6%
-5.134e-01     -4.767e-01       7.1%
 1.973e+00      2.024e+00       2.6%
-1.865e+00     -1.960e+00       5.1%
 1.380e+00      1.431e+00       3.7%
 2.135e-02      1.681e-02      21.3%
 6.187e-01      6.170e-01       0.3%
Exact        CS
             (1487 shots)
 6.918e-02      1.983e-02      71.3%
-1.364e+00     -1.227e+00      10.1%
 1.835e+00      1.654e+00       9.9%
-6.111e-01     -5.958e-01       2.5%
-5.134e-01     -4.388e-01      14.5%
 1.973e+00      2.017e+00       2.2%
-1.865e+00     -1.840e+00       1.4%
 1.380e+00      1.400e+00       1.4%
 2.135e-02      2.465e-02      15.5%
 6.187e-01      6.714e-01       8.5%
Exact        CS
             (4096 shots)
 6.918e-02      4.113e-02      40.6%
-1.364e+00     -1.389e+00       1.8%
 1.835e+00      1.982e+00       8.0%
-6.111e-01     -6.827e-

In [11]:
print(POVMPostprocessor(cs_result[4]).get_expectation_value(set_observables[0], (0,)))

0.0585330759984246
